In [1]:
print('###############################################   LAB 6   ##############################################')
print('################### Script written by Dr Alex Ter-Sarkisov@City, University of London, 2022 ############')
print('##################### DEEP LEARNING FOR SEQUENCE ANALYSIS, MSC IN ARTIFICIAL INTELLIGENCE ##############')
print('########################################################################################################')

###############################################   LAB 6   ##############################################
################### Script written by Dr Alex Ter-Sarkisov@City, University of London, 2022 ############
##################### DEEP LEARNING FOR SEQUENCE ANALYSIS, MSC IN ARTIFICIAL INTELLIGENCE ##############
########################################################################################################


In [2]:
import torch
import torch.nn as nn
from torchvision import models
import os, sys
import numpy as np
import net_ucf11
from PIL import Image as Image
from torchvision import transforms, utils, models
import pickle
import matplotlib.pyplot as plt
import torch.utils.data as data
import dataset_ucf11
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
from torch.utils.model_zoo import load_url as load_state_dict_from_url

In [3]:
device = torch.device('cpu')
if torch.cuda.is_available():
   device = torch.device('cuda')

print ('Device set to {0}'.format(device))

epoch = 10
learning_rate = 1e-6
#batch size: number of data samples in the current batch
sample_size = 2
# ResNet101
feature_extractor_base = net_ucf11.resnet101(fc_layer=False, zero_init_residual=True)
# LSTM
rnn = net_ucf11.rnn_model()

print(feature_extractor_base)
print(rnn)

Device set to cpu
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bi

In [4]:
state_dict = load_state_dict_from_url(net_ucf11.model_urls['resnet101'], progress = True)
for k,par in state_dict.items():    
    if k in feature_extractor_base.state_dict().keys() and par.size() == feature_extractor_base.state_dict()[k].size():
        feature_extractor_base.state_dict()[k].copy_(par)
        print("Copying weights into {}".format(k))
    else:
        print("Weight dimensions mismatch for {}".format(k))


Copying weights into conv1.weight
Copying weights into bn1.running_mean
Copying weights into bn1.running_var
Copying weights into bn1.weight
Copying weights into bn1.bias
Copying weights into layer1.0.conv1.weight
Copying weights into layer1.0.bn1.running_mean
Copying weights into layer1.0.bn1.running_var
Copying weights into layer1.0.bn1.weight
Copying weights into layer1.0.bn1.bias
Copying weights into layer1.0.conv2.weight
Copying weights into layer1.0.bn2.running_mean
Copying weights into layer1.0.bn2.running_var
Copying weights into layer1.0.bn2.weight
Copying weights into layer1.0.bn2.bias
Copying weights into layer1.0.conv3.weight
Copying weights into layer1.0.bn3.running_mean
Copying weights into layer1.0.bn3.running_var
Copying weights into layer1.0.bn3.weight
Copying weights into layer1.0.bn3.bias
Copying weights into layer1.0.downsample.0.weight
Copying weights into layer1.0.downsample.1.running_mean
Copying weights into layer1.0.downsample.1.running_var
Copying weights into

Copying weights into layer3.6.bn2.running_mean
Copying weights into layer3.6.bn2.running_var
Copying weights into layer3.6.bn2.weight
Copying weights into layer3.6.bn2.bias
Copying weights into layer3.6.conv3.weight
Copying weights into layer3.6.bn3.running_mean
Copying weights into layer3.6.bn3.running_var
Copying weights into layer3.6.bn3.weight
Copying weights into layer3.6.bn3.bias
Copying weights into layer3.7.conv1.weight
Copying weights into layer3.7.bn1.running_mean
Copying weights into layer3.7.bn1.running_var
Copying weights into layer3.7.bn1.weight
Copying weights into layer3.7.bn1.bias
Copying weights into layer3.7.conv2.weight
Copying weights into layer3.7.bn2.running_mean
Copying weights into layer3.7.bn2.running_var
Copying weights into layer3.7.bn2.weight
Copying weights into layer3.7.bn2.bias
Copying weights into layer3.7.conv3.weight
Copying weights into layer3.7.bn3.running_mean
Copying weights into layer3.7.bn3.running_var
Copying weights into layer3.7.bn3.weight
Co

Copying weights into layer3.18.bn3.running_var
Copying weights into layer3.18.bn3.weight
Copying weights into layer3.18.bn3.bias
Copying weights into layer3.19.conv1.weight
Copying weights into layer3.19.bn1.running_mean
Copying weights into layer3.19.bn1.running_var
Copying weights into layer3.19.bn1.weight
Copying weights into layer3.19.bn1.bias
Copying weights into layer3.19.conv2.weight
Copying weights into layer3.19.bn2.running_mean
Copying weights into layer3.19.bn2.running_var
Copying weights into layer3.19.bn2.weight
Copying weights into layer3.19.bn2.bias
Copying weights into layer3.19.conv3.weight
Copying weights into layer3.19.bn3.running_mean
Copying weights into layer3.19.bn3.running_var
Copying weights into layer3.19.bn3.weight
Copying weights into layer3.19.bn3.bias
Copying weights into layer3.20.conv1.weight
Copying weights into layer3.20.bn1.running_mean
Copying weights into layer3.20.bn1.running_var
Copying weights into layer3.20.bn1.weight
Copying weights into layer3

In [5]:
feature_extractor_base = feature_extractor_base.to(device)
rnn = rnn.to(device)

In [56]:
#load images and label
def load_data(stage, image_size = 128):
    # all data here
    if stage == "train":
        source_dir = 'ucf11_train_data/data'

    # list of folders with videos: 1,2,
    # image transform
    # Resize transform takes a Pillow image object
    transform_train_img = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])])
    # select frames
    # this will be the dimensions of one data point, 
    # seq_length x H x W
    min_frame = 0
    max_frame = 50
    seq_length = max_frame - min_frame
    params = {'batch_size':sample_size, 'shuffle':True}
    # this will return the data for ONE video
    data_point = dataset_ucf11.DatasetUCF11(source_dir, seq_length, transform_train_img)
    dataset = data.DataLoader(data_point, **params)
    return dataset

In [57]:
def train_convnet_lstm():
    # set both networks to train mode
    # get data, instantiated only once 
    train_data = load_data("train")    
    model_params = list(feature_extractor_base.parameters())+list(rnn.parameters())
    feature_extractor_base.train()
    rnn.train()
    optimizer = torch.optim.Adam(model_params, lr=learning_rate)
    # softmax + loglikelihood
    loss_function = nn.CrossEntropyLoss()
    loss_function = loss_function.to(device)
    loss_hist = []
    for e in range(epoch):
        print ('Current epoch = {0:d}'.format(e))
        total_loss = 0  
        optimizer.zero_grad()
        for idx, X, y in train_data:
            optimizer.zero_grad()            
            X, y = X.to(device), y.to(device) 
            output = rnn(feature_extractor_base(X))
            loss = loss_function(output, y.view(-1)) 
            loss_hist.append(loss)
            # increment epoch loss
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
        # average loss this epoch, divide by the number of batches in the dataset
        total_loss = total_loss/len(train_data)
        loss_hist.append(total_loss)
        print('Total loss for epoch {0:} = {1:.3f}'.format(e, total_loss))

In [ ]:
train_convnet_lstm()